In [4]:
import sys
import os

In [5]:
import pickle
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.decomposition import PCA

# Import your custom indicators
sys.path.append(os.path.abspath("../utils")) 
from indicators import calculate_RSI, calculate_MACD, calculate_SuperTrend, calculate_ATR

c:\Users\vamsi\OneDrive\Desktop\Documents\studies\complete stock market predictor\venv\Lib\site-packages\pandas_ta\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:
import pandas as pd
import numpy as np

def supertrend_direction(df, period=7, multiplier=3):
    """
    Calculate Supertrend direction: 1 for Up, -1 for Down
    
    Args:
        df (pd.DataFrame): Must contain ['high', 'low', 'close']
        period (int): ATR period
        multiplier (float): ATR multiplier
    
    Returns:
        pd.Series: 1 for Uptrend, -1 for Downtrend
    """
    df = df.copy()
    
    # 1. Calculate ATR
    high_low = df['high'] - df['low']
    high_close = abs(df['high'] - df['close'].shift(1))
    low_close = abs(df['low'] - df['close'].shift(1))
    
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(period).mean()
    
    # 2. Basic Bands
    upper_basic = (df['high'] + df['low']) / 2 + multiplier * atr
    lower_basic = (df['high'] + df['low']) / 2 - multiplier * atr
    
    # 3. Final Bands & Trend
    upper_band = upper_basic.copy()
    lower_band = lower_basic.copy()
    trend = np.zeros(len(df))
    
    for i in range(1, len(df)):
        # Update bands
        if df['close'].iloc[i-1] <= upper_band.iloc[i-1]:
            upper_band.iloc[i] = min(upper_basic.iloc[i], upper_band.iloc[i-1])
        else:
            upper_band.iloc[i] = upper_basic.iloc[i]
        
        if df['close'].iloc[i-1] >= lower_band.iloc[i-1]:
            lower_band.iloc[i] = max(lower_basic.iloc[i], lower_band.iloc[i-1])
        else:
            lower_band.iloc[i] = lower_basic.iloc[i]
        
        # Determine trend
        if df['close'].iloc[i] > upper_band.iloc[i]:
            trend[i] = 1
        elif df['close'].iloc[i] < lower_band.iloc[i]:
            trend[i] = -1
        else:
            trend[i] = trend[i-1]  # keep previous trend
    
    # First value
    trend[0] = 1 if df['close'].iloc[0] > df['close'].iloc[0] else -1
    
    return pd.Series(trend, index=df.index)

In [43]:


# -----------------------------
# 1. Load Model
# -----------------------------
def load_model(path: str):
    with open(path, "rb") as f:
        model = pickle.load(f)
    return model


# -----------------------------
# 2. Fetch Data
# -----------------------------
def get_yesterdays_data(ticker_symbol="GC=F", interval="15m", period="5d"):
    ticker = yf.Ticker(ticker_symbol)

    # Yesterday in UTC
    yesterday = (datetime.utcnow() - timedelta(days=2)).strftime("%Y-%m-%d")

    # Get data
    df = ticker.history(interval=interval, period=period)
    df = df[df.index.strftime("%Y-%m-%d") == yesterday].reset_index()

    # Lowercase column names
    df.columns = df.columns.str.lower()
    return df


# -----------------------------
# 3. Feature Engineering
# -----------------------------
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = calculate_ATR(df)
    df['Supertrend_dir'] = supertrend_direction(df, period=7, multiplier=3)
    df = calculate_RSI(df)
    df = calculate_MACD(df)
    df['time'] = pd.to_datetime(df['datetime'])
    df['Day of the week'] = df['time'].dt.dayofweek
    df['Month'] = df['time'].dt.day
    df['hour'] = df['time'].dt.hour
    df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()
    high_liquidity_hours = [13, 14, 1, 12]
    df['high_liquidity'] = df['hour'].isin(high_liquidity_hours).astype(int)
    # Drop missing values
    df.dropna(inplace=True)
    print(df.shape)
    return df


# -----------------------------
# 4. Preprocessing (Scaling + PCA)
# -----------------------------
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # Select features
    data_dropped=df.drop(["MACD_signal","MACD_diff","time"],axis=1)

    log_trns = FunctionTransformer(func=np.log1p)
    data_dropped['ATR'] = log_trns.fit_transform(data_dropped['ATR'])

    continuous_cols = ["ATR", "RSI","MACD","Day of the week","EMA_21","Month","hour"]  # example
    boolean_cols = ["Supertrend_Direction", "high_liquidity"]

    X_continuous = data_dropped[continuous_cols]
    X_boolean = data_dropped[boolean_cols]

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_continuous)

    # PCA
    pca = PCA(n_components=6)
    X_pca = pca.fit_transform(X_scaled)

    # Final dataset
    pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2","PC3","PC4","PC5","PC6"])
    final_data = pd.concat([pca_df, X_boolean.reset_index(drop=True)], axis=1)
    print(final_data.shape)
    return final_data


# -----------------------------
# 5. Predict
# -----------------------------
def make_predictions(model, processed_data):
    with open("C:/Users/vamsi/OneDrive/Desktop/Documents/studies/complete stock market predictor/Stock-Market-Predictor/project_root/models/xgb_model.pkl", "rb") as f:
        model = pickle.load(f)
    predictions = model.predict(processed_data)
    return predictions


# -----------------------------
# Main Runner (for testing now)
# -----------------------------
if __name__ == "__main__":
    model = load_model("C:/Users/vamsi/OneDrive/Desktop/Documents/studies/complete stock market predictor/Stock-Market-Predictor/project_root/models/ensemble_model.pkl")
    raw_df = get_yesterdays_data()
    feature_df = add_features(raw_df)
    processed_df = preprocess_data(feature_df)
    predictions = make_predictions(model, processed_df)



C:\Users\vamsi\AppData\Local\Temp\ipykernel_40528\3969409761.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  yesterday = (datetime.utcnow() - timedelta(days=2)).strftime("%Y-%m-%d")


KeyError: 'Low'

In [38]:
import pandas as pd
import numpy as np

def supertrend_direction(df, period=7, multiplier=3):
    """
    Calculate Supertrend direction: 1 for Up, -1 for Down
    
    Args:
        df (pd.DataFrame): Must contain ['High', 'Low', 'Close']
        period (int): ATR period
        multiplier (float): ATR multiplier
    
    Returns:
        pd.Series: 1 for Uptrend, -1 for Downtrend
    """
    df = df.copy()
    
    # 1. Calculate ATR
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(period).mean()
    
    # 2. Basic Bands
    upper_basic = (df['High'] + df['Low']) / 2 + multiplier * atr
    lower_basic = (df['High'] + df['Low']) / 2 - multiplier * atr
    
    # 3. Final Bands & Trend
    upper_band = upper_basic.copy()
    lower_band = lower_basic.copy()
    trend = np.zeros(len(df))
    
    for i in range(1, len(df)):
        # Update bands
        if df['Close'].iloc[i-1] <= upper_band.iloc[i-1]:
            upper_band.iloc[i] = min(upper_basic.iloc[i], upper_band.iloc[i-1])
        else:
            upper_band.iloc[i] = upper_basic.iloc[i]
        
        if df['Close'].iloc[i-1] >= lower_band.iloc[i-1]:
            lower_band.iloc[i] = max(lower_basic.iloc[i], lower_band.iloc[i-1])
        else:
            lower_band.iloc[i] = lower_basic.iloc[i]
        
        # Determine trend
        if df['Close'].iloc[i] > upper_band.iloc[i]:
            trend[i] = 1
        elif df['Close'].iloc[i] < lower_band.iloc[i]:
            trend[i] = -1
        else:
            trend[i] = trend[i-1]  # keep previous trend
    
    # First value
    trend[0] = 1 if df['Close'].iloc[0] > df['Close'].iloc[0] else -1
    
    return pd.Series(trend, index=df.index)